In [1]:
import cv2
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Convolution2D, Cropping2D, Flatten, Dropout, Dense, Lambda, MaxPooling2D, Activation
from keras.optimizers import RMSprop,Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
def cloning_model():
    
    model = Sequential()

    # Source:  https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf
    model = Sequential()

    model.add(Lambda(lambda x: x / 127.5 - 1.0, input_shape=(64, 64, 3)))

    # starts with five convolutional and maxpooling layers
    model.add(Convolution2D(24, 5, 5, border_mode='same', subsample=(2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

    model.add(Convolution2D(36, 5, 5, border_mode='same', subsample=(2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

    model.add(Convolution2D(48, 5, 5, border_mode='same', subsample=(2, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

    model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

    model.add(Convolution2D(64, 3, 3, border_mode='same', subsample=(1, 1)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))

    model.add(Flatten())

    # Next, five fully connected layers
    model.add(Dense(1164))
    model.add(Activation('relu'))

    model.add(Dense(100))
    model.add(Activation('relu'))

    model.add(Dense(50))
    model.add(Activation('relu'))

    model.add(Dense(10))
    model.add(Activation('relu'))

    model.add(Dense(1))

    # model.summary()

    model.compile(optimizer=Adam(1e-4), loss="mse", )

    
    return model

model =  cloning_model()

In [3]:
DIR ='./data/'

df = pd.read_csv('./data/driving_log.csv')

In [4]:
def read_img(path):
    img = cv2.imread(path)
    rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return rgb_img

In [5]:
def img_resize(img):
    ratio = img[64:130,:,:]
    res_image = cv2.resize(ratio,(64,64),interpolation=cv2.INTER_AREA)
    return res_image

In [6]:
def image_flip(image,steering):
    if random.randrange(2) == 1: 
        return cv2.flip(image,1), -1 * steering
    else: 
        return image,steering

In [7]:
def generator(data, batch_size): 
    while 1:
        bat_cnt = 0 
        X = []
        Y = []
        for line in data.iterrows():
            
            rnd = np.random.randint(0, 3)
            if rnd == 0 : 
                image = read_img(DIR + line[1].left.strip() )
                angle = line[1].steering +  0.25 
                
            elif rnd == 1:
                image = read_img(DIR + line[1].center.strip() )
                angle = line[1].steering  
                                
            else:
                image = read_img (DIR + line[1].right.strip() )
                angle = line[1].steering   - 0.25  

            image = img_resize(image)
            image, angle = image_flip(image, angle)

            X.append(image)
            Y.append(angle)
            
            bat_cnt +=1

            if bat_cnt == batch_size: 
                X_data = np.array(X)
                Y_data = np.array(Y) 
                yield (X_data, Y_data)
                bat_cnt = 0
                X = []
                Y = []

In [8]:
df = df.iloc[np.random.permutation(len(df))]
X_train, X_Validation = train_test_split(df, test_size=0.20,)

In [ ]:
early_stopping = EarlyStopping (monitor= 'val_loss',patience = 3 , verbose = 1 , mode='min')
save_weights = ModelCheckpoint('model.h5',monitor ='val_loss',save_best_only = True)

model.fit_generator(generator(X_train,128),
                    samples_per_epoch = 20096, nb_epoch =30,
                    validation_data = generator(X_Validation,64),nb_val_samples=6400,
                    callbacks = [early_stopping,save_weights]
                   )
print("Saving model weights and configuration file.")

In [13]:
import matplotlib.pyplot as plt
%matplotlib inline 
x,y = next(generator(X_train,67))

In [14]:
print(x[0].shape)
plt.imshow(x[0])

(64, 64, 3)
